# Scalar Basics in PixelPrism

This tutorial introduces the zero-dimensional **Tensor** (also known as a *scalar*)
and walks through every creation path, manipulation primitive, and arithmetic
operator available in `pixelprism.math`. Scalars form the foundation of the
symbolic math engine, so understanding them now will make vector, matrix, and
autodiff workflows far easier later on.

## Imports and helper configuration

We'll import NumPy alongside PixelPrism's tensor core, the scalar helper
factories, and both arithmetic APIs (direct `MathExpr` helpers and the functional
module).

In [1]:
import numpy as np
from pixelprism.math import utils, DType, Tensor, MathExpr
from pixelprism.math.functional import arithmetic as F

## Creating scalars with the `Tensor` constructor

The lowest-level API instantiates a `Tensor` directly. Passing Python numbers or
NumPy scalars automatically normalizes them to zero-dimensional arrays while
preserving dtype information.

In [2]:
alpha = Tensor(name="alpha", data=3.5)
beta = Tensor(name="beta", data=2, dtype=DType.FLOAT32)

print(alpha)
print(beta)
print(f"alpha.dimensions: {alpha.shape.dims}, dtype: {alpha.dtype}")
print(f"beta.dimensions: {beta.shape.dims}, dtype: {beta.dtype}")

tensor(alpha, 3.5, dtype=DType.FLOAT32, shape=())
tensor(beta, 2.0, dtype=DType.FLOAT32, shape=())
alpha.dimensions: (), dtype: DType.FLOAT32
beta.dimensions: (), dtype: DType.FLOAT32


## Scalar constructor shortcuts from `pixelprism.math.utils`

Utility helpers wrap common allocation patterns. `utils.scalar` ensures the new
tensor is zero-dimensional, while `utils.tensor` lets us wrap an existing NumPy
scalar buffer without copying.

In [4]:
gamma = utils.scalar(name="gamma", value=1.25, dtype=np.float64)
delta = utils.tensor(name="delta", data=[[3, 4], [5, 5]], mutable=False)
print(gamma)
print(delta)

tensor(gamma, 1.25, dtype=DType.FLOAT64, shape=())
ctensor(delta, [[3.0, 4.0], [5.0, 5.0]], dtype=DType.FLOAT64, shape=(2, 2))


## Inspecting metadata and mutability

Scalars expose the same metadata helpers as higher-rank tensors. Because scalars
are leaves in the computation graph, we can mutate them in-place with `set(...)`
to emulate variable updates.

In [5]:
print(f"gamma rank: {gamma.rank}, ndim: {gamma.ndim}, size: {gamma.size}")
print(f"gamma mutable: {gamma.mutable}")

print("Original gamma value:", gamma.value)
gamma.set(np.array(5.0, dtype=np.float32))
print("Updated gamma value:", gamma.value)

gamma rank: 0, ndim: 0, size: 1
gamma mutable: True
Original gamma value: 1.25
Updated gamma value: 5.0


## Arithmetic via `MathExpr` helper methods

`MathExpr.add/sub/mul/div` create symbolic nodes that connect scalars. These
nodes remember their inputs, shape, and dtype, and lazily evaluate when asked.

In [6]:
direct_ops = {
    "alpha + beta": MathExpr.add(alpha, beta),
    "alpha - beta": MathExpr.sub(alpha, beta),
    "alpha * beta": MathExpr.mul(alpha, beta),
    "alpha / beta": MathExpr.div(alpha, beta),
}

for label, node in direct_ops.items():
    print(f"{label} -> repr: {node}")
    print(f"    evaluated value: {float(node.eval())}")

alpha + beta -> repr: <MathExpr #5 add float32 () c:2>
    evaluated value: 5.5
alpha - beta -> repr: <MathExpr #6 sub float32 () c:2>
    evaluated value: 1.5
alpha * beta -> repr: <MathExpr #7 mul float32 () c:2>
    evaluated value: 7.0
alpha / beta -> repr: <MathExpr #8 div float32 () c:2>
    evaluated value: 1.75


## Arithmetic via the functional module

If you prefer functional-style helpers, `pixelprism.math.functional.arithmetic`
exposes the same operators. They build the identical expression nodes shown
above, so you can mix and match styles.

In [7]:
functional_ops = {
    "add": F.add(alpha, beta),
    "sub": F.sub(alpha, beta),
    "mul": F.mul(alpha, beta),
    "div": F.div(alpha, beta),
}

for label, node in functional_ops.items():
    print(f"Functional {label}: {node.name} -> {float(node.eval())}")

Functional add: alpha + beta -> 5.5
Functional sub: alpha - beta -> 1.5
Functional mul: alpha * beta -> 7.0
Functional div: alpha / beta -> 1.75


## Building and visualizing a scalar expression graph

Let's compose a slightly richer scalar expression that will show how nodes
compose:

$$h(Alpha, Beta, \gamma) = \frac{Alpha \times Beta}{Beta + \gamma}$$

The following helper prints a lightweight tree so we can see which scalars feed
the numerator and denominator.

In [8]:
numerator = MathExpr.mul(alpha, beta)
denominator = MathExpr.add(beta, gamma)
scalar_expr = MathExpr.div(numerator, denominator)

def render_expr(node, indent=0):
    spacer = " " * indent
    if hasattr(node, "children") and node.children:
        print(f"{spacer}- node: {node.name} (op={node.op.name}, shape={node.shape.dims})")
        for child in node.children:
            render_expr(child, indent + 4)
    else:
        value = node.value if hasattr(node, "value") else node.eval()
        print(f"{spacer}- leaf: {node.name}, value={float(value)}")
    # end if
# end render_expr

render_expr(scalar_expr)
print("Expression evaluation:", float(scalar_expr.eval()))

- node: alpha * beta / beta + gamma (op=div, shape=())
    - node: alpha * beta (op=mul, shape=())
        - leaf: alpha, value=3.5
        - leaf: beta, value=2.0
    - node: beta + gamma (op=add, shape=())
        - leaf: beta, value=2.0
        - leaf: gamma, value=5.0
Expression evaluation: 1.0


## Evaluating expressions under different values

`Tensor.set(...)` permanently mutates a scalar leaf. Alternatively, pass overrides
into `MathExpr.eval(...)` to temporarily bind new runtime values (useful for
feeding placeholders).

In [9]:
print("Baseline evaluation:", float(scalar_expr.eval()))

alpha.set(np.array(10.0, dtype=np.float32))
print("After mutating alpha with set(...):", float(scalar_expr.eval()))

custom_result = scalar_expr.eval(alpha=np.array(2.0, dtype=np.float32), beta=np.array(4.0, dtype=np.float32))
print("Temporary override via eval(...):", float(custom_result))

print("alpha remains mutated (set) value:", float(alpha.value))

Baseline evaluation: 1.0
After mutating alpha with set(...): 2.857142857142857
Temporary override via eval(...): 0.8888888888888888
alpha remains mutated (set) value: 10.0
